## Introduction to Dataset Processing
#### Carl Shan

This Jupyter Notebook will share more details about how to process your data. Data processing is like preparing the ingredients before cooking; if you prepare them poorly (e.g., leave things half-peeled and dirty) , the meal will taste poor no matter how skillful a chef you are. 

It's similarly true in machine learning. Dataset processing can be one of the most important things you can do to get your model to perform well.

#### Introducing some helpful "magic" Jupyter commands
? - this will bring up the documentation of a function

In [ ]:
import pandas as pd
from sklearn import preprocessing

%pylab inline

Download the [student performance data](http://archive.ics.uci.edu/ml/machine-learning-databases/00320/) and change the path below to wherever you put the data.

In [ ]:
student_data = pd.read_csv('../data/student/student-mat.csv', sep=';')

In [ ]:
student_data.head()

#### Converting Categorical Values to Numerical Ones

Looking at the data above, we want to convert a number of the columns from categorical to numerical. Most machine learning models deal with numbers and don't know how to model data that is in text form. As a result we need to learn how to do things such as e.g., convert the values in the `school` column to numbers.

#### First, let's see what values there are in the `school` column

In [ ]:
# This shows a list of unique values and how many times they appear
student_data['school'].value_counts()

In [ ]:
# Converting values in the school column to text
# We are going to define a function that takes a single value and apply it to all the values
def convert_school(row):
    if row == 'GP':
        return 0
    elif row == 'MS':
        return 1
    else:
        return None

#### Avoid for loops
Normally, we might write a for loop like the one below. But this is really slow when using Pandas. _Don't write loops like this_.

In [ ]:
# Here's a slow way of using the above function
%time
converted_school = []

for row in student_data['school']:
    new_value = convert_school(row)
    converted_school.append(new_value)
converted_school

#### Use apply instead of for loops
This will do the same thing as the for loop above, but _much_ faster.

In [ ]:
%time
converted_school = student_data['school'].apply(convert_school)
converted_school

#### Using sklearn's built-in preprocessing module, we can do the same thing

In [ ]:
enc_school = preprocessing.LabelEncoder()
transformed_school = enc_school.fit_transform(student_data['school'])
transformed_school

#### We can also use one-hot encoding if we have more than two values. We still need to encode it first, as we did above.
See example at https://stackoverflow.com/a/43589167/2159992

In [ ]:
enc_mjob = preprocessing.LabelEncoder()
encoded_mjob = enc_mjob.fit_transform(student_data['Mjob'])
encoded_mjob

In [ ]:
onehot_mjob = preprocessing.OneHotEncoder(sparse=False)
transformed_mjob = onehot_mjob.fit_transform(encoded_mjob.reshape(-1,1))
transformed_mjob

Once we've fitted the label encoder and one-hot encoder, we can use them to transform more values.

In [ ]:
onehot_mjob.transform(enc_mjob.transform(['other', 'health']).reshape(-1,1))

#### What if we want to apply a transform that looks at multiple values in the row?
For instance, what if we want to create a new column with a 1 if both parents have the highest level of education measured?

In [ ]:
medu_index = student_data.columns.get_loc('Medu')
fedu_index = student_data.columns.get_loc('Fedu')

def both_parents_edu(row):
    if row[medu_index] > 3 and row[fedu_index] >= 4:
        return 1
    else:
        return 0
    
# axis 1 means that we will apply the function to each row
student_data['parents_high_edu'] = student_data.apply(both_parents_edu, axis=1)
student_data.head(10)

#### Dealing with Null values
To show you how to deal with null values, I'm going to make some simulated data of students.

In [ ]:
grades = np.random.choice(range(1, 13), 100) # chooses 100 random numbers between 1 - 12
num_friends_or_none = list(range(0, 20)) + [None] * 5
num_friends = np.random.choice(num_friends_or_none, 100)
new_data = pd.DataFrame(data={'Grade': grades, '# Friends': num_friends})

In [ ]:
new_data.head(n=20)

#### One way to deal with null values is to drop them

In [ ]:
new_data['# Friends'].dropna()

#### We can also drop any rows with nulls from the entire table.

In [ ]:
new_data.dropna()

#### Or we can replace the null values with an average

In [ ]:
average_friends = new_data['# Friends'].mean()
new_data['# Friends'].fillna(average_friends)

In [ ]:
new_data['# Friends'] = new_data['# Friends'].fillna(average_friends)

#### What if instead of null values, there is something else that stands for missing values?
Try the replace function.

In [ ]:
grades = np.random.choice(range(1, 13), 100) # chooses 100 random numbers between 1 - 12
num_friends_or_none = list(range(0, 20)) + ["Unknown"] * 5
num_friends = np.random.choice(num_friends_or_none, 100)
unknown_data = pd.DataFrame(data={'Grade': grades, '# Friends': num_friends})
unknown_data

In [ ]:
unknown_data.replace("Unknown", 10)

#### Now let's learn how to standardize data
By that I mean to transform our data so that it has a mean of 0 and a standard deviation of 1

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit_transform(new_data)

#### What if we would like to split up columns?
We can use apply here, as well!

In [ ]:
grades = np.random.choice(range(1, 13), 100) # chooses 100 random numbers between 1 - 12
grades_and_ages = ['{grade} - {age}'.format(grade=grade, age=grade+6) for grade in grades]
num_friends_or_none = list(range(0, 20)) + ["Unknown"] * 5
num_friends = np.random.choice(num_friends_or_none, 100)
combined_data = pd.DataFrame(data={'Grade and Age': grades_and_ages, '# Friends': num_friends})
combined_data

In [ ]:
grade_age_index = combined_data.columns.get_loc('Grade and Age')
friends_index = combined_data.columns.get_loc('# Friends')

def split_grade_and_age(row):
    friends = row[friends_index]
    grade_age = row[grade_age_index].split(" - ")
    grade = int(grade_age[0])
    age = int(grade_age[1])
    return pd.Series({'Grade': grade, 'Age': age, '# Friends': friends})

# axis 1 means we will get the entire row at once
combined_data.apply(split_grade_and_age, axis=1)